In [55]:
from collections import defaultdict, Counter

import graphviz
import yaml
import json
import os

In [56]:
coverage_path = '../data/memdjpeg_coverage/data_14.yaml'
func_graph_path = '../data/main.dot'
mapping_path = '../data/mapping.json'

In [7]:
mapping = json.load(open(mapping_path, 'r'))

In [9]:
coverage = yaml.load(open(coverage_path, 'r'), Loader=yaml.SafeLoader)

In [11]:
covered_set = set()

In [13]:
for run, blocks in coverage.items():
    covered_set |= set(blocks)

In [14]:
print(len(covered_set))

3889


In [15]:
overlaps = Counter()

In [16]:
for function, blocklist in mapping.items():
    overlaps[function] = len(covered_set & set([int(b, 16) for b in blocklist]))

In [17]:
overlaps.most_common(10)

[(u'start_pass', 153),
 (u'_unknown', 119),
 (u'decode_mcu', 115),
 (u'decode_mcu_AC_refine', 94),
 (u'decompress_smooth_data', 94),
 (u'jpeg_make_d_derived_tbl', 90),
 (u'get_dht', 83),
 (u'get_sos', 75),
 (u'start_pass_huff_decoder', 71),
 (u'decode_mcu_AC_first', 71)]

In [18]:
sum(overlaps.values())

2948

In [87]:
address_label = {}

In [88]:
with open(func_graph_path, 'r') as in_file:
    with open('../res/main_compressed.dot', 'w') as out_file:
        
        for line in in_file.readlines():
            
            if 'node [fillcolor=gray style=filled shape=box];' in line:
                new_line = '\tnode [fillcolor=gray style=filled];\n'
            
            elif 'label=' in line:
                address_g = line.split()[0]
                address_m = line.split()[0].strip().translate(None, string.punctuation)
                content = line.split('label=')[1][:-2].replace('\l', '\n').translate(None, '"')
                address_label[address_m] = content
                new_line = line.split('label=')[0] + 'label=' + address_g + ']' + os.linesep
                            
            else:
                new_line = line
            
            out_file.write(new_line)

In [89]:
functions = [key for key in mapping.keys() if key != '_unknown']
print(len(functions))

447


In [90]:
address_label.keys()[0].strip().translate(None, string.punctuation)

'0x08049dd6'

In [91]:
address_label[address_label.keys()[0]]

'| 0x08049dd6      mov eax, dword [local_8a4h]\n| 0x08049ddd      lea edx, dword [eax + 0x150]\n| 0x08049de3      mov eax, dword [local_894h]\n| 0x08049dea      add eax, 0xa\n| 0x08049ded      mov eax, dword [eax]\n| 0x08049def      mov dword [edx], eax\n'

In [92]:
json.dump(address_label, open('../res/address_label_map.json', 'w'), indent=2)